In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Load and preprocess data
df = pd.read_excel('seat.xlsx')
df = df.dropna()

temperature = df['Temperature'].values.reshape(-1, 1)
energy_demand = df['EnergyDemand'].values.reshape(-1, 1)

scaler_temp = MinMaxScaler()
scaler_energy = MinMaxScaler()
temperature_normalized = scaler_temp.fit_transform(temperature)
energy_demand_normalized = scaler_energy.fit_transform(energy_demand)

def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

seq_length = 48  # changed (doubled)
X, y = create_sequences(temperature_normalized, seq_length)
y = energy_demand_normalized[seq_length:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# more units, extra gru layer, 2 dropouts
model = Sequential([
    GRU(100, activation='tanh', return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    GRU(50, activation='tanh'),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# changed
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)


history = model.fit(
    X_train, y_train,
    epochs=20, #changed --> increase more bc early stopping is implemented
    batch_size=64, #changed
    validation_split=0.1,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

predictions_normalized = model.predict(X_test)

predictions = scaler_energy.inverse_transform(predictions_normalized)
y_test_original = scaler_energy.inverse_transform(y_test)

mse = mean_squared_error(y_test_original, predictions)
mae = mean_absolute_error(y_test_original, predictions)
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")